<a href="https://colab.research.google.com/github/moosunny/Transformer_Translator/blob/main/Seq2Seq_Translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import torch
from torch import nn, optim
from transformers import MarianMTModel, MarianTokenizer, AutoTokenizer # MT: Machine Translation
import pandas as pd
from tqdm import tqdm
import math, random

# 한국어 토크나이저 활용(upstage)
tokenizer = AutoTokenizer.from_pretrained("upstage/solar-1-mini-tokenizer")
# 패딩을 오른쪽으로 설정
tokenizer.padding_side = "right"


# 한국어-영어 대화체 데이터 다운로드(link:)
data = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Transformer_Translation/2_대화체.xlsx')
data

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/3.51k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.31M [00:00<?, ?B/s]

,대분류,소분류,상황,Set Nr.,발화자,원문,번역문
0,비즈니스,회의,의견 교환하기,1,A-1,이번 신제품 출시에 대한 시장의 반응은 어떤가요?,How is the market's reaction to the newly rele...
1,비즈니스,회의,의견 교환하기,1,B-1,판매량이 지난번 제품보다 빠르게 늘고 있습니다.,The sales increase is faster than the previous...
2,비즈니스,회의,의견 교환하기,1,A-2,그렇다면 공장에 연락해서 주문량을 더 늘려야겠네요.,"Then, we'll have to call the manufacturer and ..."
3,비즈니스,회의,의견 교환하기,1,B-2,"네, 제가 연락해서 주문량을 2배로 늘리겠습니다.","Sure, I'll make a call and double the volume o..."
4,비즈니스,회의,의견 교환하기,2,A-1,지난 회의 마지막에 논의했던 안건을 다시 볼까요?,Shall we take a look at the issues we discusse...
...,...,...,...,...,...,...,...
99995,여행/쇼핑,쇼핑,"계산/포장/배달 (계산 장소 문의, 계산 오류 등)",24999,B-2,"저희가 가격표 배치를 잘못해서 혼동을 드렸나 봐요, 죄송해요.",It seems that we didn't place the price tags c...
99996,여행/쇼핑,쇼핑,"계산/포장/배달 (계산 장소 문의, 계산 오류 등)",25000,A-1,"백화점 포인트로 계산하고 싶은데, 가능한가요?",Can I pay using the department store points?
99997,여행/쇼핑,쇼핑,"계산/포장/배달 (계산 장소 문의, 계산 오류 등)",25000,B-1,"네, 물론이죠, 전화번호 입력해주시면 됩니다.","Yes, of course, you just need to enter your ph..."
99998,여행/쇼핑,쇼핑,"계산/포장/배달 (계산 장소 문의, 계산 오류 등)",25000,A-2,"입력했어요, 전액 백화점 포인트로 결제하고 싶어요.","I entered it, I want to pay it with all the de..."


In [3]:
eos_idx = tokenizer.eos_token_ids
pad_idx = tokenizer.pad_token_ids
unk_idx = tokenizer.unk_token_ids


# eos token과 pad 토큰의 인덱스가 동일함을 확인 -> unk 토큰을 활용한 문장 패딩 수행
print(eos_idx)
print(pad_idx)
print(unk_idx)

2
2
0


In [4]:
tokenizer.pad_token = tokenizer.unk_token
print(tokenizer.pad_token)

pad_idx = tokenizer.pad_token_id
print(pad_idx)

<unk>
0


In [5]:
# 하이퍼 파라미터 모음
max_len = 100
batch_size = 128
vocab_size = tokenizer.vocab_size # 64000개

EPOCH = 15
criterion = nn.CrossEntropyLoss(ignore_index=pad_idx)
embedding_dim = 512

In [6]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return self.data.shape[0]

    def __getitem__(self, idx):
        return self.data.loc[idx, "원문"], self.data.loc[idx, "번역문"]

custom_DS = CustomDataset(data)
print(custom_DS[0])

train_DS, val_DS, test_DS = torch.utils.data.random_split(custom_DS, [97000, 2000, 1000])
# 논문에서는 450만개 영,독 문장 pair 사용

train_DL = torch.utils.data.DataLoader(train_DS, batch_size=batch_size, shuffle=True)
val_DL = torch.utils.data.DataLoader(val_DS, batch_size=batch_size, shuffle=True)
test_DL = torch.utils.data.DataLoader(test_DS, batch_size=batch_size, shuffle=True)

print(len(train_DS))
print(len(val_DS))
print(len(test_DS))

('이번 신제품 출시에 대한 시장의 반응은 어떤가요?', "How is the market's reaction to the newly released product?")
97000
2000
1000


In [8]:
i = 5
src_text, trg_text  = test_DS[0]
print(f"인덱스:{test_DS.indices[i]}")
print(f"원문:{src_text}")
print(f"원문:{trg_text}")

인덱스:19871
원문:그럼 미안하지만 축의금 좀 대신 전해줄 수 있겠어?
원문:Then will you give her the money gift for a celebration on behalf of me?


In [9]:
eos_idx = tokenizer.eos_token_ids
print(eos_idx)

for src_text, trg_text in train_DL:
  print(src_text)
  print(trg_text)
  print(len(src_text))
  print(len(trg_text))

  src = tokenizer(src_text, padding = True, truncation=True, max_length=max_len, return_tensors = 'pt', add_special_tokens = False).input_ids
  trg_text = [s + tokenizer.eos_token for s in trg_text] # eos token 추가 필요
  trg = tokenizer(trg_text, padding = True, truncation=True, max_length=max_len, return_tensors = 'pt', add_special_tokens = True).input_ids

  print(src[:2])
  print(trg[:2])
  print(src.shape)
  print(trg.shape)

  print(trg[:, -1])
  print(tokenizer.decode(trg[trg[:,-1]==pad_idx,:][0]))
  print(trg[5, :-1])
  print(trg[5, 1:])

  break

2
('저는 제 차가 아니면 운전하기 좀 거북해서요.', '그럼 제가 팀장님께 가서 전달을 드리도록 하겠습니다.', '너희는 1차 지명한 신인을 1년 만에 버리는 팀인데?', '그러면 그레이 색상으로 교환처리 해주세요.', '회식하고 오전 회의하고 붙어있으면 꽤 신경 쓰이는 일이네요.', '그 배우가 이번에 감독이랑 싸워서 중간에 하차한대.', '담당하는 직원이 따로 있기 때문에 이곳에서는 불가능합니다.', '진심이세요? 그렇다면 정말 기분 좋네요. 좋게 봐주시니 감사해요.', '학생 한 명과 저, 그리고 어른 한 명이 더 있어요.', '어쩐지 좋은 냄새가 난다고 했는데. 맛있겠다.', '나는 그냥 너 먹고 싶은 거 먹을 테니 알아서 주문해.', '네, 제가 국내 번역 에이전시 통해서 알아보고 말씀드리겠습니다.', '메인 식사 요리는 어떤 메뉴로 준비해 드릴까요?', '매장에서 사용하는 포도씨유로 튀긴 겁니다.', '여기는 너만 온 게 아니라, 다른 사람들이 많이 있잖아.', '오늘 고치시는 분이 오셔서 보시더니 누수가 맞대.', '각각 15,000원씩이니 총 60,000원입니다.', '제가 세탁해야 하는 옷이 있는데 객실에 그냥 두면 될까요?', '아니요. 환불받고 다른 상품으로 구매하고 싶습니다.', '여기서 3층으로 가시면 바로 옆 건물 연결통로가 있어요.', '지난번처럼 샘플을 뒤섞어서 가져오면 주문이 힘들어요.', '캐리어가 1개 없어졌는데, 분실물 접수를 어디서 해야 하나요?', '그래도 그렇지, 너랑 어디 같이 못 다니겠다, 창피해서.', '나 지금 현금 없는데 만 원만 좀 빌려줘라, 내일 줄게.', '그러면 앞머리를 살짝만 넘겨 드릴게요.', '니트는 입어보실 수 없고 이 티셔츠는 입어보실 수 있습니다.', '현재 상영하고 있는 연극 정보를 어디서 얻을 수 있나요?', '이 휴대전화가 아직 약정이 만료되지 않았는데 그게 가능한가요?', '그렇군요. 다른 분에게 부탁해 보겠습니다. 조심해서 다녀오세요.', '이 사진 속 단발은 모양이 동글동글해 보

In [16]:
embedding_dim

512

In [28]:
embedding_dim = 1000 # Seq2Seq 논문 상으로 1000 Embedding_dim
enc_embedding = nn.Embedding(vocab_size, embedding_dim)
dec_embedding = nn.Embedding(vocab_size, embedding_dim)

for src, trg in train_DL:
  print(src[0]) # (배치 사이즈, 문장 길이)
  print(trg[0])
  N = len(train_DL)
  src = tokenizer(src, padding = True, truncation = True, max_length = max_len, return_tensors = 'pt', add_special_tokens = False).input_ids
  trg = [s + tokenizer.eos_token for s in trg] # eos token 추가 필요
  trg = tokenizer(trg, padding = True, truncation = True, max_length = max_len, return_tensors = 'pt', add_special_tokens = True).input_ids
  print(src.shape)
  print(trg.shape)

  ### Encoder Layer
  enc_embedded_src = enc_embedding(src)
  print(f"임베딩 후 차원: {enc_embedded_src.shape}") # (배치 사이즈, 문장 길이, 임베딩 차원)
  enc_src = nn.Dropout(0.5)(enc_embedded_src)
  print(f"임베딩 -> Dropout 후 차원: {enc_src.shape}")
  enc_lstm = nn.LSTM(input_size = embedding_dim, hidden_size = 1000, num_layers = 4, dropout = 0.5, batch_first= True)
  # encoder output
  enc_output, (enc_hidden, enc_cell) = enc_lstm(enc_src)
  print(f"LSTM 레이어 통과 후 차원: {enc_output.shape}")
   # num_layers 만큼 hidden, cell state 출력
  print(f"hidden state 차원: {enc_hidden.shape}")
  print(f"cell state 차원: {enc_cell.shape}")

  ### Decoder Layer
  # input = [batch size]
  # hidden = [n layers * n directions, batch size, hidden dim]
  # cell = [n layers * n directions, batch size, hidden dim]
  # n directions in the decoder will both always be 1, therefore:
  # hidden = [n layers, batch size, hidden dim]
  # context = [n layers, batch size, hidden dim]
  trg = trg.permute(1,0)
  dec_input =  trg[0, :]
  dec_input = dec_input.unsqueeze(0)
  print(dec_input.shape)
  # dec_embedded_trg = dec_embedding(trg)
  # print(f"임베딩 후 차원: {dec_embedded_trg.shape}") # (배치 사이즈, 문장 길이, 임베딩 차원)
  # dec_trg = nn.Dropout(0.5)(dec_embedded_trg)
  # dec_lstm = nn.LSTM(input_size = , hidden_size = 1000, num_layers = 4, dropout = 0.5, batch_first = True)



  break

고객님, 이 가격은 현재 세일 가격이 맞습니다.
Sir, this price is the correct sale price right now.
torch.Size([128, 27])
torch.Size([128, 34])
임베딩 후 차원: torch.Size([128, 27, 1000])
임베딩 -> Dropout 후 차원: torch.Size([128, 27, 1000])
LSTM 레이어 통과 후 차원: torch.Size([128, 27, 1000])
hidden state 차원: torch.Size([4, 128, 1000])
cell state 차원: torch.Size([4, 128, 1000])
torch.Size([1, 128])


In [ ]:
import random

class Encoder(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, n_layers, dropout):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=dropout)
        self.dropout = nn.Dropout(dropout)

    def forward(self, src):
        # src = [src length, batch size]
        embedded = self.dropout(self.embedding(src))
        # embedded = [src length, batch size, embedding dim]
        outputs, (hidden, cell) = self.lstm(embedded)
        # outputs = [src length, batch size, hidden dim * n directions]
        # hidden = [n layers * n directions, batch size, hidden dim]
        # cell = [n layers * n directions, batch size, hidden dim]
        # outputs are always from the top hidden layer
        return hidden, cell

class Decoder(nn.Module):
    def __init__(self, output_dim, embedding_dim, hidden_dim, n_layers, dropout):
        super().__init__()
        self.output_dim = output_dim
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.embedding = nn.Embedding(output_dim, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=dropout)
        self.fc_out = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input, hidden, cell):
        # input = [batch size]
        # hidden = [n layers * n directions, batch size, hidden dim]
        # cell = [n layers * n directions, batch size, hidden dim]
        # n directions in the decoder will both always be 1, therefore:
        # hidden = [n layers, batch size, hidden dim]
        # context = [n layers, batch size, hidden dim]
        input = input.unsqueeze(0)
        # input = [1, batch size]
        embedded = self.dropout(self.embedding(input))
        # embedded = [1, batch size, embedding dim]
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        # output = [seq length, batch size, hidden dim * n directions]
        # hidden = [n layers * n directions, batch size, hidden dim]
        # cell = [n layers * n directions, batch size, hidden dim]
        # seq length and n directions will always be 1 in this decoder, therefore:
        # output = [1, batch size, hidden dim]
        # hidden = [n layers, batch size, hidden dim]
        # cell = [n layers, batch size, hidden dim]
        prediction = self.fc_out(output.squeeze(0))
        # prediction = [batch size, output dim]
        return prediction, hidden, cell

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        assert (
            encoder.hidden_dim == decoder.hidden_dim
        ), "Hidden dimensions of encoder and decoder must be equal!"
        assert (
            encoder.n_layers == decoder.n_layers
        ), "Encoder and decoder must have equal number of layers!"

    def forward(self, src, trg, teacher_forcing_ratio):
        # src = [src length, batch size]
        # trg = [trg length, batch size]
        # teacher_forcing_ratio is probability to use teacher forcing
        # e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        batch_size = trg.shape[0]
        trg_length = trg.shape[1]
        trg_vocab_size = self.decoder.output_dim
        # tensor to store decoder outputs
        outputs = torch.zeros(trg_length, batch_size, trg_vocab_size).to(self.device)
        # last hidden state of the encoder is used as the initial hidden state of the decoder
        hidden, cell = self.encoder(src)
        # hidden = [n layers * n directions, batch size, hidden dim]
        # cell = [n layers * n directions, batch size, hidden dim]
        # first input to the decoder is the <sos> tokens
        input = trg[0, :]
        # input = [batch size]
        for t in range(1, trg_length):
            # insert input token embedding, previous hidden and previous cell states
            # receive output tensor (predictions) and new hidden and cell states
            output, hidden, cell = self.decoder(input, hidden, cell)
            # output = [batch size, output dim]
            # hidden = [n layers, batch size, hidden dim]
            # cell = [n layers, batch size, hidden dim]
            # place predictions in a tensor holding predictions for each token
            outputs[t] = output
            # decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            # get the highest predicted token from our predictions
            top1 = output.argmax(1)
            # if teacher forcing, use actual next token as next input
            # if not, use predicted token
            input = trg[t] if teacher_force else top1
            # input = [batch size]
        return outputs

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

max_len = 50

# seed 고정
def seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed(42)

# class CustomDataset(Dataset):
#     def __init__(self, text, tokenizer, vocab_size, max_length = max_len, embedding_dim=1000):
#         self.text = text
#         self.tokenizer = tokenizer
#         self.max_length = max_length
#         self.vocab_size = vocab_size
#         self.embedding_dim = embedding_dim
#         self.emb = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim)

#     def __len__(self):
#         return len(self.text)

#     def __getitem__(self, idx):
#       text = self.text[idx]

#       encoding = self.tokenizer(
#                                 text,
#                                 padding="max_length",
#                                 max_length=self.max_length,
#                                 truncation= True,
#                                 return_tensors="pt"
#                                 )
#       input_idx = encoding["input_ids"]
#       embedded_inputs = self.emb(input_idx)


#       return embedded_inputs.squeeze(0) # (1, max_length, emb_dim)




# # 패딩을 오른쪽으로 설정
# tokenizer.padding_side = "right"

# # pad token index 설정
# tokenizer.pad_token_id = 0  # 패딩 토큰의 인덱스 설정


# # source 데이터 분리
# train_src = data_kor_list[:89999]
# val_src = data_kor_list[90000:94999]
# test_src = data_kor_list[9500:]

# # target 데이터 분리
# train_target = data_eng_list[:89999]
# val_target = data_eng_list[90000:94999]
# test_target = data_eng_list[95000:]


# # 데이터셋 생성
# train_src_dataset = CustomDataset(text = train_src, tokenizer = tokenizer,  vocab_size = vocab_size)
# val_src_dataset = CustomDataset(text = val_src, tokenizer = tokenizer,  vocab_size = vocab_size)
# test_src_dataset = CustomDataset(text = test_src, tokenizer = tokenizer,  vocab_size = vocab_size)

# train_input_data = train_data


# # train dataloader
# train_src_dataloader = DataLoader(train_src_dataset, batch_size=128, shuffle=False)
# val_src_dataloader = DataLoader(val_src_dataset, batch_size=128, shuffle=False)
# test_src_dataloader = DataLoader(test_src_dataset, batch_size=128, shuffle=False)

In [ ]:
# target data
# train_target_dataset = CustomDataset(text = train_target, tokenizer = tokenizer,  vocab_size = vocab_size)
# val_target_dataset = CustomDataset(text = val_target, tokenizer = tokenizer,  vocab_size = vocab_size)
# test_target_dataset = CustomDataset(text = test_target, tokenizer = tokenizer,  vocab_size = vocab_size)

# train_target_dataloader = DataLoader(train_target_dataset, batch_size=128, shuffle=False)
# val_target_dataloader = DataLoader(val_target_dataset, batch_size=128, shuffle=False)
# test_target_dataloader = DataLoader(test_target_dataset, batch_size=128, shuffle=False)

In [ ]:
input_dim = len(tokenizer.get_vocab().items())
output_dim = len(tokenizer.get_vocab().items())
encoder_embedding_dim = 256
decoder_embedding_dim = 256
hidden_dim = 512
n_layers = 2
encoder_dropout = 0.5
decoder_dropout = 0.5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

encoder = Encoder(
    input_dim,
    encoder_embedding_dim,
    hidden_dim,
    n_layers,
    encoder_dropout,
)

decoder = Decoder(
    output_dim,
    decoder_embedding_dim,
    hidden_dim,
    n_layers,
    decoder_dropout,
)

model = Seq2Seq(encoder, decoder, device).to(device)

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)


model.apply(init_weights)

In [ ]:
print(input_dim,"input_dim")
print(output_dim,"output_dim")

64000 input_dim
64000 output_dim


In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


print(f"The model has {count_parameters(model):,} trainable parameters")

The model has 30,426,710 trainable parameters


In [ ]:
pad_index = tokenizer.pad_token_id
print(pad_index)

0


In [ ]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss(ignore_index= pad_index)

def train_fn(model, data_loader, optimizer, criterion, clip, teacher_forcing_ratio, device):
    model.train()
    epoch_loss = 0
    for i, batch in enumerate(data_loader):
        print(batch["kor_ids"].dtype)  # float32/int64/long 확인
        print(batch["kor_ids"].min(), batch["kor_ids"].max())  # 범위 체크
        print(batch["eng_ids"].min(), batch["eng_ids"].max())  # 범위 체크
        print(batch["kor_ids"].device)
        src = batch["kor_ids"].to(device).long()
        print(src.dtype)
        trg = batch["eng_ids"].to(device).long()
        print(src.shape)
        print(trg.shape)
        # src = [src length, batch size]
        # trg = [trg length, batch size]
        optimizer.zero_grad()
        output = model(src, trg, teacher_forcing_ratio)
        # output = [trg length, batch size, trg vocab size]
        output_dim = output.shape[-1]
        output = output[1:].view(-1, output_dim)
        print(output.shape)
        # output = [(trg length - 1) * batch size, trg vocab size]
        trg = trg[1:].view(-1)
        # trg = [(trg length - 1) * batch size]
        loss = criterion(output, trg)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        epoch_loss += loss.item()
    return epoch_loss / len(data_loader)

In [ ]:
def evaluate_fn(model, data_loader, criterion, device):
    model.eval()
    epoch_loss = 0
    with torch.no_grad():
        for i, batch in enumerate(data_loader):
            src = batch["kor_ids"].to(device).long()
            trg = batch["eng_ids"].to(device).long()
            print(src.shape)
            print(trg.shape)
            # src = [src length, batch size]
            # trg = [trg length, batch size]
            output = model(src, trg, 0)  # turn off teacher forcing
            # output = [trg length, batch size, trg vocab size]
            output_dim = output.shape[-1]
            output = output[1:].view(-1, output_dim)
            print(output.shape)
            # output = [(trg length - 1) * batch size, trg vocab size]
            trg = trg[1:].view(-1)
            # trg = [(trg length - 1) * batch size]
            loss = criterion(output, trg)
            epoch_loss += loss.item()
    return epoch_loss / len(data_loader)

In [ ]:
from tqdm.notebook import tqdm

n_epochs = 10
clip = 1.0
teacher_forcing_ratio = 0.5

best_valid_loss = float("inf")

for epoch in tqdm(range(n_epochs)):
    train_loss = train_fn(
        model,
        train_loader,
        optimizer,
        criterion,
        clip,
        teacher_forcing_ratio,
        device,
    )
    valid_loss = evaluate_fn(
        model,
        val_loader,
        criterion,
        device,
    )
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), "tut1-model.pt")
    print(f"\tTrain Loss: {train_loss:7.3f} | Train PPL: {np.exp(train_loss):7.3f}")
    print(f"\tValid Loss: {valid_loss:7.3f} | Valid PPL: {np.exp(valid_loss):7.3f}")

  0%|          | 0/10 [00:00<?, ?it/s]

torch.int64
tensor(0) tensor(62733)
tensor(0) tensor(28808)
cpu


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
